<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  4 in 1  Link 使用 dash

In [ ]:
import dash
from dash import html, dcc, Input, Output
from flask import send_from_directory
import os

# 圖片基礎資料夾（外部目錄）
IMAGE_ROOT = '/path/to/folder'  # ⬅️ 這裡換成你的資料夾路徑

app = dash.Dash(__name__)
server = app.server

# 🔧 自定義圖片提供端點
@server.route("/images/<path:subpath>")
def serve_image(subpath):
    folder, filename = os.path.split(subpath)
    return send_from_directory(os.path.join(IMAGE_ROOT, folder), filename)

# 主頁面 Layout，使用 dcc.Location 來讀網址參數
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='image-layout')  # 圖片排版區
])

@app.callback(
    Output('image-layout', 'children'),
    Input('url', 'search')
)
def display_images(search):
    from urllib.parse import parse_qs

    # ✅ 解析網址參數
    query = parse_qs(search.lstrip('?'))
    chart_name = query.get('chart_name', [''])[0]
    create = query.get('create', [''])[0]

    if not chart_name or not create:
        return html.Div("請在網址中提供 chart_name 和 create 參數")

    # ✅ 建構搜尋資料夾路徑
    folder_path = os.path.join(IMAGE_ROOT, create)
    if not os.path.exists(folder_path):
        return html.Div(f"資料夾不存在: {folder_path}")

    # ✅ 搜尋符合的圖片
    files = os.listdir(folder_path)
    target_images = [f for f in files if f.startswith(chart_name) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # ✅ 定位每個位置的圖片
    positions = {
        'cpk': None,
        'mean': None,
        'sigma': None,
        'trend': None
    }

    for fname in target_images:
        lname = fname.lower()
        if 'cpk' in lname and positions['cpk'] is None:
            positions['cpk'] = fname
        elif 'mean' in lname and positions['mean'] is None:
            positions['mean'] = fname
        elif 'sigma' in lname and positions['sigma'] is None:
            positions['sigma'] = fname
        elif 'trend' in lname and positions['trend'] is None:
            positions['trend'] = fname

    def img_or_blank(key):
        if positions[key]:
            src = f"/images/{create}/{positions[key]}"
            return html.Img(src=src, style={'width': '100%', 'maxHeight': '300px'})
        else:
            return html.Div(style={'height': '300px', 'width': '100%'})  # 空白佔位

    # ✅ 2x2 排列
    return html.Div([
        html.Div([
            html.Div(img_or_blank('cpk'), style={'width': '48%', 'margin': '1%'}),
            html.Div(img_or_blank('mean'), style={'width': '48%', 'margin': '1%'})
        ], style={'display': 'flex', 'justifyContent': 'space-between'}),
        html.Div([
            html.Div(img_or_blank('sigma'), style={'width': '48%', 'margin': '1%'}),
            html.Div(img_or_blank('trend'), style={'width': '48%', 'margin': '1%'})
        ], style={'display': 'flex', 'justifyContent': 'space-between'})
    ])

if __name__ == '__main__':
    app.run_server(debug=True)


## 生成ppt

In [ ]:
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

# 下載 PPT 的 callback
@app.callback(
    Output("download-data", "data"),
    Input("download-button", "n_clicks"),
    State("my-table", "derived_virtual_data"),
    prevent_initial_call=True
)
def download_ppt(n_clicks, filtered_data):
    if not filtered_data:
        return dash.no_update

    prs = Presentation()
    slide_layout = prs.slide_layouts[5]  # 空白版面
    slide = prs.slides.add_slide(slide_layout)

    title_shape = slide.shapes.title
    if title_shape:
        title_shape.text = "篩選後資料簡報"

    # 將資料整理成文字放入簡報的 textbox 裡 (這裡示範簡單放文字)
    left = Inches(0.5)
    top = Inches(1.5)
    width = Inches(9)
    height = Inches(5)

    txBox = slide.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame
    tf.word_wrap = True

    # 依欄位名稱產生標題列
    columns = filtered_data[0].keys()
    header = "\t".join(columns)
    p = tf.add_paragraph()
    p.text = header
    p.font.bold = True
    p.font.size = Pt(14)
    p.alignment = PP_ALIGN.LEFT

    # 資料每列加入一行
    for row in filtered_data:
        line = "\t".join(str(row[col]) for col in columns)
        p = tf.add_paragraph()
        p.text = line
        p.font.size = Pt(12)
        p.alignment = PP_ALIGN.LEFT

    # 把 ppt 存到記憶體 buffer
    ppt_io = BytesIO()
    prs.save(ppt_io)
    ppt_io.seek(0)

    return dcc.send_bytes(ppt_io.read(), "filtered_data.pptx")


### ppt 範例 (先測試，再放入Dash)

In [ ]:
import pandas as pd
from pptx import Presentation
from pptx.util import Inches, Pt
import os

# 模擬 chart 名單
df = pd.DataFrame({
    'chart': ['chartA', 'chartB']
})

# 建立簡報
prs = Presentation()

# 圖片位置設定（左上、右上、左下、右下）
positions = [
    (Inches(0.5), Inches(1.5)),   # 1 - 左上
    (Inches(5.0), Inches(1.5)),   # 2 - 右上
    (Inches(0.5), Inches(4.5)),   # 3 - 左下
    (Inches(5.0), Inches(4.5)),   # 4 - 右下
]

img_width = Inches(4.5)
img_height = Inches(3)

for chart_name in df['chart']:
    slide_layout = prs.slide_layouts[5]  # 空白
    slide = prs.slides.add_slide(slide_layout)

    # 加入 chart 標題
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = chart_name
    title_frame.paragraphs[0].font.size = Pt(28)

    # 加入圖片（有就放，沒有就跳過）
    for i in range(1, 5):
        img_path = f"./images/{chart_name}_{i}.png"
        if os.path.exists(img_path):
            left, top = positions[i - 1]
            slide.shapes.add_picture(img_path, left, top, width=img_width, height=img_height)
        else:
            # 空白：不放圖
            print(f"👉 沒找到圖片，略過: {img_path}")

# 儲存簡報
prs.save("charts_with_missing_images_ok.pptx")
